In [0]:
# import pyspark
# import glow

In [0]:
# builder = pyspark.sql.SparkSession.builder.appName("GlowVCFExplore") \
#     .config("spark.jars.packages", "io.projectglow:glow-spark3_2.12:1.2.1") \
#     .config("spark.hadoop.io.compression.codecs", "io.projectglow.sql.util.BGZFCodec")

In [0]:
# spark = builder.getOrCreate()

In [0]:
# spark = glow.register(spark)

In [0]:
# spark

In [0]:
# Page Break

# Explore bcBio Somatic VCF

In [0]:
%sh
mkdir /dbfs/tmp/dannywong
wget -O /dbfs/tmp/dannywong/na12878-na24385-somatic-hg38-truth.vcf.gz https://s3.amazonaws.com/bcbio/giab/NA12878-NA24385/2018-07-05/na12878-na24385-somatic-hg38-truth.vcf.gz
wget -O /dbfs/tmp/dannywong/na12878-na24385-germline-hg38-truth.vcf.gz https://s3.amazonaws.com/bcbio/giab/NA12878-NA24385/2018-07-05/na12878-na24385-germline-hg38-truth.vcf.gz

In [0]:
# bcbio_somatic_src = "./data/bcbio_giab_somatic/na12878-na24385-somatic-hg38-truth.vcf.gz"
bcbio_somatic_src = "dbfs:/tmp/dannywong/na12878-na24385-somatic-hg38-truth.vcf.gz"

In [0]:
bcbio_somatic_df = spark.read.format("vcf").load(bcbio_somatic_src)

In [0]:
bcbio_somatic_df.printSchema()

In [0]:
bcbio_somatic_df.createOrReplaceTempView("bcbio_somatic_table")

In [0]:
spark.sql("describe bcbio_somatic_table").show(n=1000, truncate=True)

In [0]:
%sql
DESCRIBE EXTENDED bcbio_somatic_table

In [0]:
spark.sql("select count(1) as number_of_variants from bcbio_somatic_table").show()

In [0]:
spark.sql("select distinct contigName from bcbio_somatic_table order by contigName").show(46)

In [0]:
spark.sql("select contigName, start, end from bcbio_somatic_table").show()

In [0]:
spark.sql("select contigName, count(end) as num_of_pos from bcbio_somatic_table group by contigName order by num_of_pos desc").show(46)

In [0]:
spark.sql("select referenceAllele, alternateAlleles, array_size(alternateAlleles) from bcbio_somatic_table").show() #Need runtime 10.5 to run array_size - https://docs.databricks.com/release-notes/runtime/10.5.html

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_snps \
from bcbio_somatic_table \
where \
    char_length(referenceAllele) = 1 and \
    array_size(alternateAlleles) = 1 and \
    char_length(alternateAlleles[0]) = 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_snps desc").show()

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_deletions \
from bcbio_somatic_table \
where \
    char_length(referenceAllele) > 1 and \
    array_size(alternateAlleles) = 1 and \
    char_length(alternateAlleles[0]) = 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_deletions desc").show()

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_insertions \
from bcbio_somatic_table \
where \
    char_length(referenceAllele) = 1 and \
    array_size(alternateAlleles) > 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_insertions desc").show()

In [0]:
spark.sql("select count(1) as `A>C` from bcbio_somatic_table \
where \
    referenceAllele = 'A' and \
    alternateAlleles[0] = 'C' and \
    array_size(alternateAlleles) = 1").show()

In [0]:
spark.sql("select count(1) as `A>AT` from bcbio_somatic_table \
where \
    referenceAllele = 'A' and \
    alternateAlleles[0] = 'AT' and \
    array_size(alternateAlleles) = 2").show()

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId, genotypes.alleleDepths \
from bcbio_somatic_table").show(truncate=False)

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId from bcbio_somatic_table \
where contigName = 'chr1' and end = 852047").show(truncate=False)

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId[0] from bcbio_somatic_table \
where genotypes.sampleId[0] = 'NA12878'").show(truncate=False)

In [0]:
# Page Break

# Stop Spark Session

In [0]:
# spark.stop()

In [0]:
# Continue to next notebook